In [3]:
from lark import InlineTransformer
from lark import Lark

In [34]:
grammar = r"""

    ?start : pipe
    
    ?atom : SIGNED_NUMBER -> number
    
    ?expr : expr SUM_OP atom -> sum_op
          | atom
        
    ?pipe : expr "|>" pipe
          | expr
    
    SUM_OP : /[+-]/
    %import common.SIGNED_NUMBER
    %import common.WS
    %ignore WS

"""

In [35]:
class PipelineTransformer(InlineTransformer):
    number = float
        
    def pipe(self, expr_left, expr_right):  
        return (expr_left, expr_right)
    
    def sum_op(self, left, op, right):
        return (str(op), left, right)

In [36]:
pipeline_parser = Lark(grammar, parser="lalr", transformer=PipelineTransformer())
pipeline = pipeline_parser.parse

In [37]:
res = pipeline("1+1 |> 1-9 |> 1+9")

In [38]:
res

(('+', 1.0, 1.0), (('-', 1.0, 9.0), ('+', 1.0, 9.0)))

In [50]:
head, args = ((1,2),(2,3),(9,6))

ValueError: too many values to unpack (expected 2)

(2, 3)

In [41]:
def eval_expr(expr):
    if isinstance(expr, float):
        return expr
    
    head, *args = expr
    
    print("------")
    print(expr)
    print("HEAD:", head,"ARGS:", args)
    if isinstance(head, tuple):
        return eval_expr(head) + eval_expr(args)
    elif head == '+':
        x, y = args
        return eval_expr(x) + eval_expr(y)
    elif head == '-':
        x, y = args
        return eval_expr(x) - eval_expr(y)

    else:
        raise ValueError('argumento inválido para S-expression: %r' % head)

In [42]:
eval_expr(res)

------
(('+', 1.0, 1.0), (('-', 1.0, 9.0), ('+', 1.0, 9.0)))
HEAD: ('+', 1.0, 1.0) ARGS: [(('-', 1.0, 9.0), ('+', 1.0, 9.0))]
------
('+', 1.0, 1.0)
HEAD: + ARGS: [1.0, 1.0]
------
[(('-', 1.0, 9.0), ('+', 1.0, 9.0))]
HEAD: (('-', 1.0, 9.0), ('+', 1.0, 9.0)) ARGS: []
------
(('-', 1.0, 9.0), ('+', 1.0, 9.0))
HEAD: ('-', 1.0, 9.0) ARGS: [('+', 1.0, 9.0)]
------
('-', 1.0, 9.0)
HEAD: - ARGS: [1.0, 9.0]
------
[('+', 1.0, 9.0)]
HEAD: ('+', 1.0, 9.0) ARGS: []
------
('+', 1.0, 9.0)
HEAD: + ARGS: [1.0, 9.0]


ValueError: not enough values to unpack (expected at least 1, got 0)